In [1]:
import pandas as pd
import numpy as np

# Ler e Mudar nomes

In [2]:

#dfDelta=pd.read_excel("Stocks_fev.xlsx", sheet_name="Sheet1")

dfNinjas=pd.read_excel("D:\\Brands and Ninjas\\Delta\\Mar-abril23\\CafesAldi.xlsx")

#dfSonae=pd.read_excel("Sellout_fev.xlsx")


dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE"})
#dfSonae = dfSonae.rename(columns={'Data': 'DATA', "LojaEntreposto":"STORE", "DscArtigo":"DESC_ARTIGO"})

# VER

In [4]:
#dfDelta.head()
dfNinjas.head()
#dfSonae.head()

,Hora,DATA,Nome da Loja,STORE,DeltaQ XL QALIDUS (40 CAPS),DeltaQ XL QHARACTER (40 CAPS),DeltaQ XL AQTIVUS (40 CAPS),DeltaQ XL MYTHIQ (40 CAPS),DELTA Q deQafeinatus Cx. 10UN,Lote Chávena MQ 250g,CAFFE GRÃO CHAVENA 1KG,Cafés Solúvel 200g
0,17:19:43,2023-03-31,ALDI AGUALVA-CACÉM,79.0,1,1,1,1,1,1,1,1
1,15:12:36,2023-03-31,ALDI ALCABIDECHE,NaN,1,1,0,1,1,1,1,1
2,14:36:18,2023-04-01,ALDI ALCANTARILHA,66.0,0,1,1,0,1,1,1,1
3,18:17:26,2023-04-01,ALDI ALCOBAÇA,43.0,1,1,0,1,1,1,1,1
4,16:55:00,2023-03-31,ALDI ALCOCHETE,NaN,0,1,1,1,1,1,1,1


# Mudar para datetime os que forem precisos e remover L's e 0's

In [3]:
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'])
dfSonae['DATA'] = pd.to_datetime(dfSonae['DATA'])

In [4]:
dfSonae['STORE'] = dfSonae['STORE'].str.replace('L', '')
dfSonae['STORE'] = dfSonae['STORE'].astype('int')

# Pedir dados fundamentais

In [24]:
x= int(input("Número de colunas que não são produto: "))
y= int(input("A partir de qual coluna aparece produto: "))
n=len(dfNinjas.columns)-x

nomesProd=dfNinjas.columns[y-1:y-1+n]
nomesResto=dfNinjas.columns.difference(nomesProd)

Número de colunas que não são produto: 5
A partir de qual coluna aparece produto: 5


# Código Ninjas (pôr long)

In [6]:
dfNinjasLong = dfNinjas.melt(id_vars=nomesResto, value_vars=nomesProd, var_name='DESC_ARTIGO', value_name='NinjaInfo')

# Código para Delta (Stocks)

In [7]:
dfMeio= pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE","DESC_ARTIGO"]) 
dfMeio=dfMeio.drop(columns=["ar_stor","STORE_NAME","Enc PBL + DIRECTO"])

# Código para Sonae (Sellouts)

In [10]:
dfFinal= pd.merge(dfMeio, dfSonae, how="left", on = ["DATA","STORE","DESC_ARTIGO"])
dfFinal=dfFinal.drop(columns=["Unnamed: 3","Ean","Loja_y"])
dfFinal=dfFinal.rename(columns={'Loja_x': 'Loja'})

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts

## Fazer:

- Avaliação da informação ninja X stock

In [22]:
dfFinal["Estado"]= pd.Series(dtype='int')

dfFinal['Estado'] = np.where(dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), 1,
                         np.where(~dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), 2,
                                  np.where(dfFinal['NinjaInfo'].astype(bool) & ~dfFinal['SOH'].astype(bool), 3, 4)))

# 1: Há Stock e Sinal Positivo Ninja
# 2: Há Stock e Sinal Negativo Ninja
# 3: Não há Stock e Sinal Positivo Ninja
# 4: Não há Stock e Sinal Negativo Ninja

# Reomear e organizar

In [25]:
dfFinal.head() #O que é o Artigo?

,DATA,Hora da visita,Loja,Quantidade de produtos Delta presentes,STORE,DESC_ARTIGO,NinjaInfo,ARTIGO,SOH,INTRANSIT,EXPECTED,PRES_STOCK,VND (D-1),EAN,Quant,Valor,ValorOnline,Estado
0,2023-02-25,09:59:00,Continente Amadora,5,2,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,133.0,0.0,60.0,6.0,7.0,5.601082e+12,11.0,227.59,0.00,1
1,2023-02-24,13:17:00,Continente Antas,5,14,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,100.0,0.0,0.0,6.0,1.0,5.601082e+12,1.0,20.69,0.00,1
2,2023-02-26,19:29:00,Continente Aveiro,3,458,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,40.0,0.0,6.0,6.0,4.0,5.601082e+12,4.0,82.76,20.69,1
3,2023-02-25,19:58:00,Continente Barreiro,2,927,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,0,7632237.0,27.0,0.0,0.0,6.0,2.0,5.601082e+12,1.0,20.69,0.00,2
4,2023-02-25,18:59:00,Continente Beja,5,207,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,235.0,0.0,12.0,6.0,10.0,5.601082e+12,13.0,268.97,0.00,1


In [30]:
dfFinal=dfFinal.rename(columns={"VND (D-1)": "1_Dia_Antes", "Quant": "Sellout_actual"})
dfFinalCorr=dfFinal.iloc[:, [0,1,2,4,3,5,13,7,6,8,9,10,11,14,12,15,16,17]]

# Escrever

In [44]:
#dfFinalCorr.to_excel('ficheiro_NSS.xlsx', index=False)
#dfFinalCorr.to_csv('ficheiro_NSS.csv', index=False)